In [ ]:
import os
import re
import pandas as pd

def levenshtein_distance(s1, s2):
    if len(s1) < len(s2):
        return levenshtein_distance(s2, s1)
    if len(s2) == 0:
        return len(s1)
    previous_row = list(range(len(s2) + 1))
    for i, c1 in enumerate(s1):
        current_row = [i + 1]
        for j, c2 in enumerate(s2):
            insertions = previous_row[j + 1] + 1
            deletions = current_row[j] + 1
            substitutions = previous_row[j] + (c1 != c2)
            current_row.append(min(insertions, deletions, substitutions))
        previous_row = current_row
    return previous_row[-1]

def levenshtein_distance_words(s1, s2):
    words1 = s1.split()
    words2 = s2.split()
    if len(words1) < len(words2):
        return levenshtein_distance_words(s2, s1)
    if len(words2) == 0:
        return len(words1)
    previous_row = list(range(len(words2) + 1))
    for i, w1 in enumerate(words1):
        current_row = [i + 1]
        for j, w2 in enumerate(words2):
            insertions = previous_row[j + 1] + 1
            deletions = current_row[j] + 1
            substitutions = previous_row[j] + (w1 != w2)
            current_row.append(min(insertions, deletions, substitutions))
        previous_row = current_row
    return previous_row[-1]

def normalized_levenshtein(s1, s2):
    if not s1 and not s2:
        return 0.0
    if " " in s1 or " " in s2:
        dist = levenshtein_distance_words(s1, s2)
        max_len = max(len(s1.split()), len(s2.split()))
    else:
        dist = levenshtein_distance(s1, s2)
        max_len = max(len(s1), len(s2))
    return dist / max_len if max_len else 0.0

def evaluate_annotations(predictions, references):
    TP_exact, TP_partial, FP, FN = 0, 0, 0, 0
    partial_distances = []
    matched_preds = set()
    matched_refs = set()

    for i, pred in enumerate(predictions):
        matched = False
        for j, ref in enumerate(references):
            if ref == pred:
                TP_exact += 1
                matched_preds.add(i)
                matched_refs.add(j)
                matched = True
                break
        if not matched:
            for j, ref in enumerate(references):
                if j in matched_refs:
                    continue
                if pred in ref or ref in pred:
                    TP_partial += 1
                    matched_preds.add(i)
                    matched_refs.add(j)
                    partial_distances.append(normalized_levenshtein(pred, ref))
                    break

    FP = len(predictions) - len(matched_preds)
    FN = len(references) - len(matched_refs)
    return {
        'TP_exact': TP_exact,
        'TP_partial': TP_partial,
        'FP': FP,
        'FN': FN,
        'partial_distances': partial_distances
    }

# Ruta base
base_dir = '/content/drive/My Drive/data_eval'
os.makedirs(base_dir, exist_ok=True)

resultados = []
global_counts = {'TP_exact': 0, 'TP_partial': 0, 'FP': 0, 'FN': 0, 'partial_distances': []}

for i in range(1, 41):
    carpeta = f'articulo_{i}'
    carpeta_path = os.path.join(base_dir, carpeta)
    if not os.path.isdir(carpeta_path):
        print(f"⚠️ Carpeta no encontrada: {carpeta_path}")
        continue

    path_expert = os.path.join(carpeta_path, 'terminos_validados_todos.txt')
    path_model = os.path.join(carpeta_path, 'terminos_extraidos_con_patrones_ con_ejemplos_mistral.txt')

    if not os.path.exists(path_expert) or not os.path.exists(path_model):
        print(f"⚠️ Archivos faltantes en {carpeta}")
        continue

    with open(path_expert, 'r', encoding='utf-8') as f:
        expert_terms = [line.strip().lower() for line in f if line.strip()]

    with open(path_model, 'r', encoding='utf-8') as f:
        candidate_terms = [line.strip().lower() for line in f if line.strip()]

    r = evaluate_annotations(candidate_terms, expert_terms)
    TP_total = r['TP_exact'] + r['TP_partial']
    precision = TP_total / (TP_total + r['FP']) if TP_total + r['FP'] > 0 else 0.0
    recall = TP_total / (TP_total + r['FN']) if TP_total + r['FN'] > 0 else 0.0
    f1 = 2 * precision * recall / (precision + recall) if precision + recall > 0 else 0.0
    f2 = 5 * precision * recall / (4 * precision + recall) if precision + recall > 0 else 0.0
    avg_lev = sum(r['partial_distances']) / len(r['partial_distances']) if r['partial_distances'] else None

    resultados.append({
        'Artículo': carpeta,
        'TP_exact': r['TP_exact'],
        'TP_partial': r['TP_partial'],
        'FP': r['FP'],
        'FN': r['FN'],
        'Precision': precision,
        'Recall': recall,
        'F1': f1,
        'F2': f2,
        'Avg_Norm_Levenshtein': avg_lev
    })

    for k in ['TP_exact', 'TP_partial', 'FP', 'FN']:
        global_counts[k] += r[k]
    global_counts['partial_distances'].extend(r['partial_distances'])

TP_total = global_counts['TP_exact'] + global_counts['TP_partial']
FP = global_counts['FP']
FN = global_counts['FN']
precision = TP_total / (TP_total + FP) if TP_total + FP > 0 else 0.0
recall = TP_total / (TP_total + FN) if TP_total + FN > 0 else 0.0
f1 = 2 * precision * recall / (precision + recall) if precision + recall > 0 else 0.0
f2 = 5 * precision * recall / (4 * precision + recall) if precision + recall > 0 else 0.0
avg_lev = sum(global_counts['partial_distances']) / len(global_counts['partial_distances']) if global_counts['partial_distances'] else None

ruta_resultados_por_articulo = os.path.join(base_dir, 'evaluacion_anotaciones_por_articulo_mistral_one.csv')
ruta_resultados_global = os.path.join(base_dir, 'evaluacion_anotaciones_global_mistral_one.csv')

try:
    df_resultados = pd.DataFrame(resultados)
    df_resultados.to_csv(ruta_resultados_por_articulo, index=False, encoding='utf-8')
    print(f"📄 Resultados por artículo guardados en: {ruta_resultados_por_articulo}")
except Exception as e:
    print(f"❌ Error al guardar resultados por artículo: {e}")

try:
    df_global = pd.DataFrame([{
        'TP_exact': global_counts['TP_exact'],
        'TP_partial': global_counts['TP_partial'],
        'FP': FP,
        'FN': FN,
        'Precision': precision,
        'Recall': recall,
        'F1': f1,
        'F2': f2,
        'Avg_Norm_Levenshtein': avg_lev
    }])
    df_global.to_csv(ruta_resultados_global, index=False, encoding='utf-8')
    print(f"📄 Resultados globales guardados en: {ruta_resultados_global}")
except Exception as e:
    print(f"❌ Error al guardar resultados globales: {e}")

print("✅ Evaluación completada.")


📄 Resultados por artículo guardados en: /content/drive/My Drive/data_eval/evaluacion_anotaciones_por_articulo_mistral_one.csv
📄 Resultados globales guardados en: /content/drive/My Drive/data_eval/evaluacion_anotaciones_global_mistral_one.csv
✅ Evaluación completada.
